Required Imports

In [4]:
import pandas as pd
import numpy as np
import requests
import json

Setting up Server IP and Ports along with required Data for example

In [5]:
SERVER_IP = "127.0.0.1"
pp_PORT = 5000
tb_PORT = 5001

col_names = ["VB","smcAC","smcDC","vib_table","vib_spindle","AE_table","AE_spindle"]
target = "VB"
df = pd.read_csv(r"C:\Users\chh-st\Documents\GitHub\fabos-base-service-ai-pipeline\06_Test_Framework\tests\tests\WZ_Verschleiss.csv",
                header = None,
                names = col_names,
                skiprows=1,
                index_col= False,									
                sep = ',')

model_name = "use_case_test_model"
test_size = 0.7
model = "reg"
max_trials = 5
epochs = 10
print(df.head(5))

    VB     smcAC     smcDC  vib_table  vib_spindle  AE_table  AE_spindle
0  0.0 -0.017090  0.625000   0.078125     0.314941  0.087280    0.103760
1  0.0  0.263672  0.810547   0.085449     0.301514  0.098267    0.123291
2  0.0  0.207520  0.781250   0.078125     0.303955  0.092163    0.104980
3  0.0  0.302734  0.849609   0.073242     0.300293  0.095215    0.111084
4  0.0  0.239258  1.098633   0.083008     0.299072  0.083008    0.092163


Pre-Processing Data before calling ai-toolbox service

In [6]:
# Which columns to process
col = ["smcAC", "smcDC", "vib_table", "vib_spindle", "AE_table", "AE_spindle"]

# Set parameters
my_param = {"col": col}

### DATA IN PANDAS DF ###
files = {
    'data': ("data", df.to_json(), 'application/json')
}
r = requests.post('http://{0}:{1}/nan'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
# print(resp_df.head(1))
        
# Call preprocessing service - standardize values in columns "col"
files = {
    'data': ("data", resp_df.to_json(), 'application/json')
}
r = requests.post('http://{0}:{1}/std'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
# print(resp_df.head(1))

# Call preprocessing service - Remove strongly correlating features
files = {
    'data': ("data", resp_df.to_json(), 'application/json')
}        
r = requests.post('http://{0}:{1}/rm_corr'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
print(resp_df.head(5))

    VB     smcAC  vib_spindle  AE_spindle
0  0.0  0.061127    -1.042969   -1.518555
1  0.0  0.182129    -1.161133   -1.369141
2  0.0  0.157959    -1.139648   -1.508789
3  0.0  0.198975    -1.171875   -1.461914
4  0.0  0.171631    -1.182617   -1.607422


Splitting data for model Training

In [9]:
from sklearn.model_selection import train_test_split

metrics = ["mse", "mae", "mape" ]

# Set parameters
my_param = {"model": model, "target": target, "test_size": test_size, "col": df.columns.drop("VB").values, "max_trials": max_trials, "epochs": epochs, "metric": metrics}

tmp_df = resp_df
y = tmp_df[target].values
X = tmp_df.drop(target, axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state = 42) 

X_train_lists = X_train.tolist()
y_train_lists = y_train.tolist()

X_train_json_str = json.dumps(X_train_lists)
y_train_json_str = json.dumps(y_train_lists)
        
files = {
    'X_train': ("X_train", X_train_json_str, 'application/json'),
    'y_train': ("y_train", y_train_json_str, 'application/json')
}

Call ai-toolbox service

In [13]:
r = requests.post('http://{0}:{1}/automl'.format(SERVER_IP, tb_PORT), params=my_param, files=files)  
print(r)

<Response [200]>


The zippped model is stored in the zipped model folder so as to be able to transfer that file between services

The zipped file can be extracted as it has the same name as the model name varaiable

In [15]:
with open(model_name + ".zip", 'wb') as s:
    s.write(r.content)

print("Done")

Done
